# 4 - RF model with activations

- categories: [jupyter, RF, random forest, activations]

## DATASETS:
(a) Carbonic Anhydrase II (ChEMBL205), a protein lyase,  
(b) Cyclin-dependent kinase 2 (CHEMBL301), a protein kinase,  
(c) ether-a-go-go-related gene potassium channel 1 (HERG) (CHEMBL240), a voltage-gated ion channel,  
(d) Dopamine D4 receptor (CHEMBL219), a monoamine GPCR,  
(e) Coagulation factor X (CHEMBL244), a serine protease,  
(f) Cannabinoid CB1 receptor (CHEMBL218), a lipid-like GPCR and  
(g) Cytochrome P450 19A1 (CHEMBL1978), a cytochrome P450.  
The activity classes were selected based on data availability and as representatives of therapeutically important target classes or as anti-targets.

In this last notebook we will use the activations and fingerprints created from the previous notebooks and run the same Random Forest classifier on the new dataset. 

In [1]:
#hide
top_mcc_scores = {
    
    'CHEMBL205': 0.862,
    'CHEMBL301': 0.926,
    'CHEMBL240': 0.884,
    'CHEMBL219': 0.887,
    'CHEMBL244': 0.983,
    'CHEMBL218': 0.941,
    'CHEMBL1978': 0.904}

In [2]:
#hide
# Import
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
#hide
path = Path('dataset/13321_2017_226_MOESM1_ESM/')
#df = pd.read_csv('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl.csv', index_col=0)

In [4]:
#hide
#df.head()
list(path.iterdir())

[PosixPath('dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl_ecfp_1024.csv'),
 PosixPath('dataset/13321_2017_226_MOESM1_ESM/CHEMBL205_cl.csv'),
 PosixPath('dataset/13321_2017_226_MOESM1_ESM/chembl205-data-with-ecfp-activations.csv'),
 PosixPath('dataset/13321_2017_226_MOESM1_ESM/mol_images')]

# Create Dataframe from csv file

In [5]:
dataset='chembl205'

In [6]:
df = pd.read_csv(path/f'{dataset}-data-with-ecfp-activations.csv')

In [7]:
df.head()

,CID,SMILES,Activity,ECFP4_1,ECFP4_2,ECFP4_3,ECFP4_4,ECFP4_5,ECFP4_6,ECFP4_7,...,act_502,act_503,act_504,act_505,act_506,act_507,act_508,act_509,act_510,act_511
0,CHEMBL188002,S(=O)(=O)(N)c1cc(N/C(/S)=N\c2cc(C(=O)[O-])c(cc...,1,0,0,0,0,0,0,0,...,98.090538,-40.475765,-55.758678,-40.196342,-21.565161,-39.386112,88.342987,-77.520561,30.948782,-48.423111
1,CHEMBL364127,Clc1ccc(cc1)C(=O)NC1Cc2cc(S(=O)(=O)N)ccc2C1,1,0,0,0,0,0,0,0,...,105.425598,-53.179356,-1.957846,-152.047241,-124.394402,-114.585510,123.068222,-48.347355,94.419861,-130.973587
2,CHEMBL1683469,S(=O)(=O)(N)c1ccc(cc1)CNS(=O)(=O)CC12CCC(CC1=O...,1,0,0,0,0,0,0,0,...,102.208374,-64.332008,33.721348,-192.948685,-178.112000,-143.599503,147.062531,-47.969799,126.356018,-158.886963
3,CHEMBL52564,Oc1ccccc1\C=C\C(=O)[O-],1,0,0,0,0,0,0,0,...,105.915039,-39.527615,-23.374825,-81.049446,-53.409107,-57.353622,98.793694,-46.956039,50.408783,-67.285126
4,CHEMBL21427,OB(O)c1ccc(OC)cc1,1,0,0,0,0,0,0,0,...,127.697937,-41.359352,-46.951527,-80.075348,-57.078045,-62.758518,107.616997,-67.371101,54.477600,-71.601402


# Train test split

In [8]:
#hide
from sklearn.model_selection import train_test_split, KFold

In [9]:
#hide
df.is_valid.value_counts()

False    13455
True      4486
Name: is_valid, dtype: int64

In [10]:
train = df.loc[df.is_valid==False]
test = df.loc[df.is_valid==True]

In [11]:
#hide
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13455 entries, 1 to 17940
Columns: 1541 entries, CID to act_511
dtypes: bool(1), float64(512), int64(1025), object(3)
memory usage: 158.2+ MB


In [12]:
#hide
X_train, y_train = train.drop(["CID", "SMILES", "Activity", "Image"], axis=1), train["Activity"]
X_test, y_test = test.drop(["CID", "SMILES", "Activity", "Image"], axis=1), test["Activity"]

In [13]:
#hide
y_test.count()

4486

In [14]:
#hide
y_train.count(), X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13455 entries, 1 to 17940
Columns: 1537 entries, ECFP4_1 to act_511
dtypes: bool(1), float64(512), int64(1024)
memory usage: 157.8 MB


(13455, None)

# Random Forest

In [15]:
#hide
from sklearn.ensemble import RandomForestClassifier

In [16]:
#hide
from sklearn.metrics import auc,roc_auc_score,recall_score,precision_score,f1_score
from  sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

In [17]:
# train method for Random Forest
def train_rf(X_train, X_test, y_train, y_test, n_estimators=5, criterion='gini', max_features='log2'):

    
    rf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, min_samples_split=2, max_features=max_features, 
                               max_leaf_nodes=None,bootstrap=False,oob_score=False, n_jobs=-1, random_state=69)
    
    rf.fit(X_train,y_train)
    y_pred= rf.predict(X_test)
    y_pred_prob=rf.predict_proba(X_test)
    
    temp=[]
    for j in range(len(y_pred_prob)):
        temp.append(y_pred_prob[j][1])
    auc=roc_auc_score(np.array(y_test),np.array(temp))
    acc2=accuracy_score(y_test,y_pred)
    mcc=matthews_corrcoef(y_test,y_pred)
    Recall=recall_score(y_test, y_pred,pos_label=1)
    Precision=precision_score(y_test, y_pred,pos_label=1)
    F1_score=f1_score(y_test, y_pred,pos_label=1)

    return auc,acc2,mcc,Recall,Precision,F1_score

## Train RF with settings grid from paper

In [18]:
#hide
from sklearn.model_selection import ParameterGrid

In [19]:
param_grid = {
    'n_estimators': [10,50,100,200,300,700], 
    'criterion': ['gini', 'entropy'],
    'max_features': ['log2', 'sqrt']}

param_grid = ParameterGrid(param_grid)

In [28]:
MCC=[]

In [29]:
for setting in param_grid:
    print(f"Testing combination: {setting}")
    
    
    aucs, accs, mccs, recalls, precs, f1_scores = [], [], [], [], [], []
   
        
    auc,acc2,mcc,recall,precision,F1_score = train_rf(X_train, X_test, y_train, y_test, 
                                                             n_estimators=setting['n_estimators'], 
                                                             criterion=setting['criterion'],
                                                             max_features=setting['max_features'])
        
    aucs.append(auc)
    accs.append(acc2)
    mccs.append(mcc)
    MCC.append(mcc)
    recalls.append(recall)
    precs.append(precision)
    f1_scores.append(F1_score)
    
    print(f"Average Matthews correlation: {np.mean(mccs)}")
   
    print()

Testing combination: {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 10}
Average Matthews correlation: 0.8135612528114817

Testing combination: {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 50}
Average Matthews correlation: 0.808070667802557

Testing combination: {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 100}
Average Matthews correlation: 0.8057158482003463

Testing combination: {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 200}
Average Matthews correlation: 0.8057158482003463

Testing combination: {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 300}
Average Matthews correlation: 0.8033780300912903

Testing combination: {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 700}
Average Matthews correlation: 0.8057158482003463

Testing combination: {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 10}
Average Matthews correlation: 0.8085109587994693

Testing combination: {'criterion': 'gi

The mean score of the Matthews correlation from all the different testing combinations.

In [30]:
np.mean(MCC)

0.8076599473510552

In [31]:
np.amax(MCC), top_mcc_scores['CHEMBL205']

(0.8135612528114817, 0.862)